In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [13]:
import bilby
from gwpy.timeseries import TimeSeries
import numpy as np
import ptemcee

In [3]:
logger = bilby.core.utils.logger
outdir = "GW150914-outdir"
label = "GW150914"


In [4]:
trigger_time = 1126259462.4
detectors = ["H1", "L1"]
maximum_frequency = 512
minimum_frequency = 20
roll_off = 0.4  # Roll off duration of tukey window in seconds, default is 0.4s
duration = 4  # Analysis segment duration
post_trigger_duration = 2  # Time between trigger time and end of segment
end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

In [5]:
psd_duration = 32 * duration
psd_start_time = start_time - psd_duration
psd_end_time = start_time

In [6]:
ifo_list = bilby.gw.detector.InterferometerList([])
for det in detectors:
    logger.info("Downloading analysis data for ifo {}".format(det))
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    data = TimeSeries.fetch_open_data(det, start_time, end_time)
    ifo.strain_data.set_from_gwpy_timeseries(data)

    logger.info("Downloading psd data for ifo {}".format(det))
    psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time)
    psd_alpha = 2 * roll_off / duration
    psd = psd_data.psd(
        fftlength=duration, overlap=0, window=("tukey", psd_alpha), method="median"
    )
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psd.frequencies.value, psd_array=psd.value
    )
    ifo.maximum_frequency = maximum_frequency
    ifo.minimum_frequency = minimum_frequency
    ifo_list.append(ifo)

19:49 bilby INFO    : Downloading analysis data for ifo H1
19:49 bilby INFO    : Downloading psd data for ifo H1
19:52 bilby INFO    : Downloading analysis data for ifo L1
19:52 bilby INFO    : Downloading psd data for ifo L1


In [7]:
logger.info("Saving data plots to {}".format(outdir))
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)
ifo_list.plot_data(outdir=outdir, label=label)

19:54 bilby INFO    : Saving data plots to GW150914-outdir
19:54 bilby INFO    : Generating frequency domain strain from given time domain strain.
19:54 bilby INFO    : Applying a tukey window with alpha=0.1, roll off=0.2
19:54 bilby INFO    : Generating frequency domain strain from given time domain strain.
19:54 bilby INFO    : Applying a tukey window with alpha=0.1, roll off=0.2


In [8]:
priors = bilby.gw.prior.BBHPriorDict(filename="GW150914.prior")

priors["geocent_time"] = bilby.core.prior.Uniform(
    trigger_time - 0.1, trigger_time + 0.1, name="geocent_time"
)

In [9]:
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={
        "waveform_approximant": "IMRPhenomXPHM",
        "reference_frequency": 50,
    },
)

19:54 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [10]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator,
    priors=priors,
    time_marginalization=True,
    phase_marginalization=False,
    distance_marginalization=False,
)

In [11]:
sampler = bilby.sampler.PTMCMCSampler(likelihood, priors, outdir=outdir, label=label, 
                                      nwalkers=64, ntemps=8, 
                                      adapt=True, adaptation_time=10, adaptation_lag=100, Tmax=np.inf)

SamplerNotInstalledError: Sampler PTMCMCSampler is not installed on this system

In [ ]:
nwalkers = 64
ntemps = 8
ndim = len(priors)
sampler = ptemcee.Sampler(nwalkers, ndim, )